In [ ]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.2-cp310-cp310-linux_x86_64.whl size=582719 sha256=9a1b1f2dff64ccea56b0a94f65044674b8a60dbf092c7888b7e31ced1fdc8a22
  Stored in directory: /root/.cache/pip/wheels/7a/d9/59/473fa56df8e39430eeda369500b4e7127f5b243ba24c3c4297
Successfully built annoy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random

from annoy import AnnoyIndex
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/CMPE 256 Project/tracks_features.csv')

data['id'] = data.index
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']
features = ['acousticness','danceability', 'energy', 'instrumentalness', 'liveness','speechiness', 'valence']
data_features = data[features].values

In [ ]:
data_features

array([[2.61e-02, 4.70e-01, 9.78e-01, ..., 3.56e-01, 7.27e-02, 5.03e-01],
       [1.29e-02, 5.99e-01, 9.57e-01, ..., 1.55e-01, 1.88e-01, 4.89e-01],
       [2.34e-02, 3.15e-01, 9.70e-01, ..., 1.22e-01, 4.83e-01, 3.70e-01],
       ...,
       [3.04e-02, 7.85e-01, 7.96e-01, ..., 6.64e-02, 5.64e-02, 4.67e-01],
       [6.97e-05, 6.65e-01, 8.56e-01, ..., 1.17e-01, 4.09e-02, 2.27e-01],
       [1.68e-02, 7.36e-01, 7.08e-01, ..., 2.79e-01, 5.39e-02, 2.04e-01]])

In [ ]:
data.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,0,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,2,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,3,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,4,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [ ]:
scaler = StandardScaler()
scaled_data_features = scaler.fit_transform(data_features)

In [ ]:
n_trees = 10
n_dimensions = len(features)
index = AnnoyIndex(n_dimensions, 'angular')

for i, feature_vector in enumerate(scaled_data_features):
    index.add_item(i, feature_vector)

index.build(n_trees)

True

In [ ]:
n_neighbors = 20
nearest_neighbors = {}
for i in range(len(scaled_data_features)):
    nearest_neighbors[i] = index.get_nns_by_item(i, n_neighbors)

In [ ]:
track_id_to_index = {track_id: index for index, track_id in enumerate(data['id'])}
index_to_track_id = {index: track_id for track_id, index in track_id_to_index.items()}

In [ ]:
# Create a list of the columns of having track features
import csv
track_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']
track_features_dict={}
for i, row in data.iterrows():
    track_id = row['id']
    features = {}
    for feature_name in track_features:
        features[feature_name] = row[feature_name]
    track_features_dict[track_id] = features

In [ ]:
# Function to generate a random playlist using AnnoyIndex
def generate_playlist(data, track_id_to_index, index_to_track_id, track_features_dict):
    seed_track_id = random.choice(data['id'].values)
    playlist_length = random.randint(5, 20)

    seed_index = track_id_to_index[seed_track_id]
    playlist_indices = [seed_index]

    playlist = []
    for i in range(playlist_length):
        track_id = index_to_track_id[playlist_indices[i]]
        features = track_features_dict[track_id]
        playlist.append({'id': track_id, **features})
        if i < playlist_length - 1:
            last_index = playlist_indices[-1]
            # Choose the first nearest neighbor that is not already in the playlist
            for neighbor in nearest_neighbors[last_index]:
                if neighbor not in playlist_indices:
                    next_index = neighbor
                    break
            playlist_indices.append(next_index)

    return playlist

In [ ]:
# Generate 50,000 playlists
playlists = []
for i in range(500):
    playlist = generate_playlist(data, track_id_to_index, index_to_track_id, track_features_dict)
    #print(playlist)
    playlists.append(playlist)

In [ ]:
playlists[0]

[{'id': 406430,
  'acousticness': 0.138,
  'danceability': 0.757,
  'energy': 0.157,
  'instrumentalness': 2.82e-06,
  'liveness': 0.0833,
  'speechiness': 0.0769,
  'valence': 0.743},
 {'id': 1150593,
  'acousticness': 0.0489,
  'danceability': 0.77,
  'energy': 0.13,
  'instrumentalness': 0.000117,
  'liveness': 0.0819999999999999,
  'speechiness': 0.0595,
  'valence': 0.816},
 {'id': 1150601,
  'acousticness': 0.0384,
  'danceability': 0.7559999999999999,
  'energy': 0.12,
  'instrumentalness': 0.00181,
  'liveness': 0.0526,
  'speechiness': 0.0558,
  'valence': 0.882},
 {'id': 104244,
  'acousticness': 0.091,
  'danceability': 0.746,
  'energy': 0.093,
  'instrumentalness': 0.0,
  'liveness': 0.0451,
  'speechiness': 0.0589,
  'valence': 0.866},
 {'id': 519310,
  'acousticness': 0.191,
  'danceability': 0.6629999999999999,
  'energy': 0.18,
  'instrumentalness': 0.000204,
  'liveness': 0.0862,
  'speechiness': 0.0546,
  'valence': 0.794},
 {'id': 775736,
  'acousticness': 0.225,
  

In [ ]:
# Calculate the mean feature values for each playlist
playlist_features = []
for i, playlist in enumerate(playlists, start=1):
    playlist_id = str(i)
    features = {}
    track_ids=[]
    for j in range(len(playlists[i-1])):
      track_ids.append(playlists[i-1][j]['id'])
    #print(len(track_ids))
    for feature_name in track_features:
        feature_values = [track[feature_name] for track in playlist]
        mean_feature_value = sum(feature_values) / len(feature_values)
        features[feature_name] = mean_feature_value
    
    playlist_features.append({'playlist_id': playlist_id,'track_ids': track_ids, **features})

In [ ]:
#playlist_features[1]

In [ ]:
# Save the playlist features to a CSV file
with open('/content/drive/Shareddrives/CMPE 256 Project/playlist_features_500.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['playlist_id', 'track_ids'] + track_features)
    writer.writeheader()
    for playlist in playlist_features:
        print(playlist)
        writer.writerow(playlist)

{'playlist_id': '1', 'track_ids': [406430, 1150593, 1150601, 104244, 519310, 775736, 679466, 20776], 'acousticness': 0.14728750000000002, 'danceability': 0.7168749999999999, 'energy': 0.144125, 'instrumentalness': 0.0009472275, 'liveness': 0.08489999999999998, 'speechiness': 0.06839999999999999, 'valence': 0.8111249999999999}
{'playlist_id': '2', 'track_ids': [620293, 811465, 865733, 900584, 279418, 287321, 1018761, 823165, 341151, 833168, 45589, 334484, 1155125, 33885, 1140057, 1140216, 1157441], 'acousticness': 0.01394870588235294, 'danceability': 0.8405882352941179, 'energy': 0.6448823529411765, 'instrumentalness': 1.6688235294117649e-06, 'liveness': 0.24847058823529408, 'speechiness': 0.2604705882352941, 'valence': 0.5331764705882353}
{'playlist_id': '3', 'track_ids': [1179481, 1069694, 930786, 789560, 197803, 928617, 630843, 606285, 716481, 370784], 'acousticness': 0.00813909999999999, 'danceability': 0.5863, 'energy': 0.7846, 'instrumentalness': 0.0032275449999999996, 'liveness':

In [ ]:
#### Updated Code for Generating the random playlist Using ------ANNOYINDEX------------ 
import random
import csv

# Function to generate a random playlist using AnnoyIndex and using this as a pre-processing step
def generate_playlist(data, track_id_to_index, index_to_track_id):
    seed_track_id = random.choice(data['id'].values)
    playlist_length = random.randint(5, 20)

    seed_index = track_id_to_index[seed_track_id]
    playlist_indices = [seed_index]

    for i in range(playlist_length - 1):
        last_index = playlist_indices[-1]
        # Choose the first nearest neighbor that is not already in the playlist
        for neighbor in nearest_neighbors[last_index]:
            if neighbor not in playlist_indices:
                next_index = neighbor
                break
        playlist_indices.append(next_index)

    # Convert the integer indices back to track IDs
    playlist_track_ids = [index_to_track_id[index] for index in playlist_indices]

    return playlist_track_ids

# Generate 50,000 playlists
playlists = []
for i in range(50):
    playlist = generate_playlist(data, track_id_to_index, index_to_track_id)
    playlists.append(playlist)

# Save the playlists to a CSV file with their playlist IDs
with open('/content/drive/Shareddrives/CMPE 256 Project/playlists.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['playlist_id', 'id'])
    for i, playlist in enumerate(playlists, start=1):
        for track_ids in playlist:
          writer.writerow([str(i), str(track_ids)])

In [ ]:
##Usage of using the seek number or selecting the first song for curaing the plalist for th
seed_track_id = random.choice(data['id'].values) # Replace this with the id of your seed track
playlist_length = 10

seed_index = track_id_to_index[seed_track_id]
playlist_indices = [seed_index]

for i in range(playlist_length - 1):
    last_index = playlist_indices[-1]
    # Choose the first nearest neighbor that is not already in the playlist
    for neighbor in nearest_neighbors[last_index]:
        if neighbor not in playlist_indices:
            next_index = neighbor
            break
    playlist_indices.append(next_index)

# Convert the integer indices back to track IDs
playlist_track_ids = [index_to_track_id[index] for index in playlist_indices]

print("Generated Playlist:")
for i, track_id in enumerate(playlist_track_ids, start=1):
    print(f"{i}. {track_id}")

Generated Playlist:
1. 874544
2. 642758
3. 227773
4. 269065
5. 228127
6. 543512
7. 261658
8. 368111
9. 276023
10. 262778


#### Transition Smoothness

In [ ]:
def pairwise_cosine_similarity(playlist, data, data_features):
    similarities = []
    for i in range(len(playlist) - 1):
        track_1_id = playlist[i]
        track_2_id = playlist[i + 1]
        
        track_1_index = data[data['id'] == track_1_id].index[0]
        track_2_index = data[data['id'] == track_2_id].index[0]
        
        track_1_features = data_features[track_1_index]
        track_2_features = data_features[track_2_index]
        
        similarity = cosine_similarity([track_1_features], [track_2_features])[0][0]
        similarities.append(similarity)
    return similarities

In [ ]:
similarities = pairwise_cosine_similarity(playlist_track_ids, data, scaled_data_features)

NameError: ignored

In [ ]:
average_similarity = np.mean(similarities)
print("Average Cosine Similarity:", average_similarity)

In [ ]:
# Assuming playlist_track_ids contains the track IDs of the generated playlist
playlist_indices = [track_id_to_index[track_id] for track_id in playlist_track_ids]

#### Intra-list Diversity 

In [ ]:
def intra_list_diversity(playlist, features):
    pairwise_distances = cosine_distances(features[playlist])
    avg_distance = np.mean(pairwise_distances)
    return avg_distance

In [ ]:
# Calculate Intra-list diversity
diversity_score = intra_list_diversity(playlist_indices, scaled_data_features)
print(f"Intra-list Diversity: {diversity_score}")

#### Artist diversity

In [ ]:
def artist_diversity(playlist, track_to_artist):
    unique_artists = len(set([track_to_artist[track] for track in playlist]))
    artist_diversity_score = unique_artists / len(playlist)
    return artist_diversity_score

In [ ]:
track_to_artist = dict(zip(data['id'], data['artist_ids']))
artist_diversity_score = artist_diversity(playlist_track_ids, track_to_artist)
print(f"Artist Diversity: {artist_diversity_score}")